# Basics & Prereqs (run once)

If you don't already have the downloaded dependencies; if you don't have TheMovieDB data indexed run this

In [1]:
from ltr import download, index
download.run(); index.run()

GET https://dl.bintray.com/o19s/RankyMcRankFace/com/o19s/RankyMcRankFace/0.1.1/RankyMcRankFace-0.1.1.jar
GET http://es-learn-to-rank.labs.o19s.com/tmdb.json
Done.


## Switch to Solr
By default examples run against elastic, the following snippet will change things to Solr

In [1]:
from ltr import useSolr
useSolr()

Switched to Solr client_mode


# Our Task: Optimizing "Drama" and "Science Fiction" queries

In this example we have two user queries

- Drama
- Science Fiction

And we want to train a model to return the best movies for these movies when a user types them into our search bar.

We learn through analysis that searchers prefer newer science fiction, but older drama. Like a lot of search relevance problems, two queries need to be optimized in *different* directions

### Synthetic Judgment List Generation

To setup this example, we'll generate a judgment list that rewards new science fiction movies as more relevant; and old drama movies as relevant.

In [2]:
from ltr import date_genre_judgments
judgments = date_genre_judgments.buildJudgments(judgmentsFile='data/genre_by_date_judgments.txt')

Generating judgments for scifi & drama movies
Done


In [2]:
# Uncomment this line to see the judgments
# 
# for judgment in judgments:
#    print(judgment.toRanklibFormat())

### Feature selection should be *easy!*

Notice we have 4 proposed features, that seem like they should work! This should be a piece of cake...

1. Release Year of a movie `release_year` - feature ID 1
2. Is the movie Science Fiction `is_scifi` - feature ID 2
3. Is the movie Drama `is_drama` - feature ID 3
4. Does the search term match the genre field `is_genre_match` - feature ID 4


In [3]:
config = [
            {
                "store": "genre", # Note: This overrides the _DEFAULT_ feature store location
                "name" : "release_year",
                "class" : "org.apache.solr.ltr.feature.SolrFeature",
                "params" : {
                  "q" : "{!func}def(release_year,2000)"
                }
            },
            {
                "store": "genre",
                "name" : "is_sci_fi",
                "class" : "org.apache.solr.ltr.feature.SolrFeature",
                "params" : {
                  "q" : "genres:\"Science Fiction\"^=10.0"
                }
            },
            {
                "store": "genre",
                "name" : "is_drama",
                "class" : "org.apache.solr.ltr.feature.SolrFeature",
                "params" : {
                  "q" : "genres:\"Drama\"^=4.0"
                }
            },
            {
                "store": "genre",
                "name" : "is_genre_match",
                "class" : "org.apache.solr.ltr.feature.SolrFeature",
                "params" : {
                  "q" : "genres:\"${keywords}\"^=100.0"
                }
            }
]


from ltr import setup_ltr
setup_ltr.run(config=config, featureset='genre')

Deleted classic model: 200
Deleted genre model: 200
Deleted latest model: 200
Delete _DEFAULT feature store: 200
Delete genre feature store: 200
Delete release feature store: 200
Created genre feature store under tmdb: 200


### Log from search engine -> to training set

Each feature is a query to be scored against the judgment list

In [4]:
from ltr import collectFeatures
trainingSet = collectFeatures.trainingSetFromJudgments(judgmentInFile='data/genre_by_date_judgments.txt', 
                                                       trainingOutFile='data/genre_by_date_judgments_train.txt', 
                                                       featureSet='genre')

Recognizing 2 queries...


KeyError: 'id'

### Training - Guaraneed Perfect Search Results!

We'll train a LambdaMART model against this training data.

In [2]:
from ltr import train
trainLog = train.run(trainingInFile='data/genre_by_date_judgments_train.txt',
                     metric2t='NDCG@10',
                     featureSet='genre',
                     modelName='genre')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
print("Perfect NDCG! {}".format(trainLog.rounds[-1]))

Running java -jar data/RankyMcRankFace.jar -ranker 6 -metric2t NDCG@10 -tree 100 -train data/genre_by_date_judgments_train.txt -save data/genre_model.txt
DONE
Delete model genre: 200
Created model genre: 201

Impact of each feature on the model
1 - 2026215.8362666676
2 - 1656425.5189779215
3 - 0.009307155595000033
4 - 0.0
Perfect NDCG! 1.0


### But this search sucks!
Try searches for "Science Fiction" and "Drama"

In [3]:
from ltr import search
search.run(keywords="Science Fiction", modelName="genre")

{"size": 5, "query": {"sltr": {"params": {"keywords": "Science Fiction"}, "model": "genre"}}}
Elstree Calling 
10.404307 
1930 
['Music', 'Comedy'] 
A series of 19 musical and comedy "vaudeville" sketches presented in the form of a live broadcast hosted by Tommy Handley (as himself). There are two "running gags" which connect the sketches. In one, an actor wants to perform Shakespeare, but he is continually denied air-time. The other gag has an inventor trying to view the broadcast on television. 
---------------------------------------
With Byrd at the South Pole 
10.404307 
1930 
['Documentary'] 
With Byrd at the South Pole (1930) is a documentary film about Rear Admiral Richard E. Byrd and his 1st quest to the South Pole beginning at the Little America-Exploration Base. The film's soundtrack consists mostly of music and sound effects, with narration read by Floyd Gibbons. The film won at the 3rd Academy Awards for Best Cinematography. 
---------------------------------------
Liliom 

### Why didn't it work!?!? Training data

1. Examine the training data, do we cover every example of a BAD result
2. Examine the feature impacts, do any of the features the model uses even USE the keywords?

### Ranklib only sees the data you give it, we don't have good enough coverage

You need to have feature coverage, especially over negative examples. Most documents in the index are negative! 

One trick commonly used is to treat other queries positive results as this queries negative results. Indeed what we're missing here are negative examples for "Science Fiction" that are not science fiction movies. A glaring omission, we'll handle now... With the `autoNegate` flag, we'll add additional negative examples to the judgment list

In [4]:
from ltr import date_genre_judgments, collectFeatures
date_genre_judgments.buildJudgments(judgmentsFile='data/genre_by_date_judgments.txt',
                                    autoNegate=True)

collectFeatures.trainingSetFromJudgments(judgmentInFile='data/genre_by_date_judgments.txt', 
                                         trainingOutFile='data/genre_by_date_judgments_train.txt', 
                                         featureSet='genre')

from ltr import train
trainLog = train.run(trainingInFile='data/genre_by_date_judgments_train.txt',
                     metric2t='NDCG@10',
                     featureSet='genre',
                     modelName='genre')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
print("Perfect NDCG! {}".format(trainLog.rounds[-1]))

Generating judgments for scifi & drama movies
Done
Recognizing 2 queries...
REBUILDING TRAINING DATA for Science Fiction (0/2)
REBUILDING TRAINING DATA for Drama (1/2)
Running java -jar data/RankyMcRankFace.jar -ranker 6 -metric2t NDCG@10 -tree 100 -train data/genre_by_date_judgments_train.txt -save data/genre_model.txt
DONE
Delete model genre: 200
Created model genre: 201

Impact of each feature on the model
4 - 802616758.725657
1 - 89257217.35776828
3 - 1573733.5830246205
2 - 1.953984578809708e-20
Perfect NDCG! 1.0


### Now try those queries...

Replace keywords below with 'science fiction' or 'drama' and see how it works

In [5]:
from ltr import search
search.run(keywords="Drama", modelName="genre")

{"size": 5, "query": {"sltr": {"params": {"keywords": "Drama"}, "model": "genre"}}}
The Shamrock Handicap 
9.445964 
1926 
['Romance', 'Drama'] 
The first film having an Irish motif that John Ford directed, a six reel delight set in Eire's County Kildare and in the United States, with a steeplechase background, mixing charged elements of comedy and sentimental drama, benefiting from a sterling cast including Leslie Fenton, Janet Gaynor, and Ford favourite J. Farrell MacDonald. 
---------------------------------------
Sparrows 
9.445964 
1926 
['Drama'] 
Evil Mr.Grimes keeps a rag-tag bunch orphans on his farm deep in a swamp in the US South. He forces them to work in his garden and treats them like slaves. They are watched over by the eldest, Molly. A gang in league with Mr. Grimes kidnaps Doris, the beautiful little daughter of a rich man, and hides her out on Grimes' farm, awaiting ransom. When the police close in, and Mr. Grimes threatens to throw Doris into the bottomless mire, Mol

### The next problem

- Overfit to these two examples
- We need many more queries, covering more use cases